In [11]:

import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
    

In [37]:

file_path = "data/DataTransacciones_PruebasIngreso.xlsx"

# Cargar el archivo de Excel
xls = pd.ExcelFile(file_path)

# Leer las hojas como strings
comercios = pd.read_excel(xls, sheet_name="comercios", dtype=str)
transacciones = pd.read_excel(xls, sheet_name="transacciones", dtype=str)
    

In [38]:
transacciones


,Id_comercio,mes-anio transaccion,cantidad transacciones
0,003QDVFJCJ,3-2021,5
1,003QDVFJCJ,4-2021,9
2,003QDVFJCJ,5-2021,16
3,008LJH6YHN,2-2021,2
4,008LJH6YHN,3-2021,5
...,...,...,...
8352,ZZMG2NE0Q6,4-2021,1
8353,ZZMG2NE0Q6,5-2021,1
8354,ZZWN25XK8J,3-2021,24
8355,ZZWN25XK8J,4-2021,8


In [39]:
comercios

,Id_comercio,Fecha vinculación datafono,Fecha de creacion comercio,Categoria comercio,Sub Categoria comercio,Id_terminal,Ciudad,Fecha 1ra_transaccion,Monto ventas Acumulado,Monto ventas ultimo mes
0,003QDVFJCJ,"mar. 26, 2021 12:40pm","mar. 26, 2021 12:17pm",Comercio detallista,Ropa / Accesorios,01299,Bogotá D.C.,"mar. 26, 2021 12:42pm",6542000,1240000
1,008LJH6YHN,"feb. 10, 2021 3:31pm","feb. 10, 2021 3:15pm",Belleza y cuidado personal,Spa,00591,Bogotá D.C.,"feb. 10, 2021 3:34pm",4380000,1102000
2,00B3DGV62G,"mar. 3, 2021 3:26pm","mar. 3, 2021 3:09pm",Comercio detallista,Auto partes,02130,Bogotá D.C.,"mar. 3, 2021 3:30pm",11892500,1297000
3,00ILW1BB3Y,"mar. 24, 2021 12:45pm","mar. 24, 2021 12:33pm",Comercio detallista,Tienda naturista,00662,Bogotá D.C.,"mar. 24, 2021 12:51pm",372500,74500
4,01AK7UX4UU,"mar. 30, 2021 1:50pm","mar. 30, 2021 1:40pm",Comercio detallista,Tecnología,01784,Mosquera,"mar. 30, 2021 1:52pm",165500,0
...,...,...,...,...,...,...,...,...,...,...
3066,ZYU8UUM32R,"mar. 11, 2021 11:24am","mar. 11, 2021 11:14am",Comercio detallista,Tecnología,02829,Bogotá D.C.,"mar. 11, 2021 11:30am",26110920,4912000
3067,ZYV1ZCZ7IM,"mar. 25, 2021 2:46pm","mar. 25, 2021 2:36pm",Servicios,Tecnología y software,01348,Bogotá D.C.,"mar. 25, 2021 2:47pm",4400500,491000
3068,ZZJ3KWF9VC,"mar. 19, 2021 4:09pm","mar. 19, 2021 4:00pm",Alimentos y bebidas,Restaurante / Café,02938,Bogotá D.C.,"mar. 19, 2021 4:12pm",65100,20000
3069,ZZMG2NE0Q6,"mar. 12, 2021 11:05am","mar. 12, 2021 10:49am",Comercio detallista,Elementos de seguridad,01908,Bogotá D.C.,"mar. 12, 2021 11:06am",300000,149000


In [40]:

# Procesar los datos
comercios["Monto ventas Acumulado"] = pd.to_numeric(comercios["Monto ventas Acumulado"])
comercios["Monto ventas ultimo mes"] = pd.to_numeric(comercios["Monto ventas ultimo mes"]) 
transacciones["cantidad transacciones"] =pd.to_numeric(transacciones["cantidad transacciones"])

In [41]:
comercios = comercios.merge(
    transacciones.groupby("Id_comercio", as_index=False)["cantidad transacciones"].sum(),
    on="Id_comercio",
    how="left"
)

In [42]:
comercios

,Id_comercio,Fecha vinculación datafono,Fecha de creacion comercio,Categoria comercio,Sub Categoria comercio,Id_terminal,Ciudad,Fecha 1ra_transaccion,Monto ventas Acumulado,Monto ventas ultimo mes,cantidad transacciones
0,003QDVFJCJ,"mar. 26, 2021 12:40pm","mar. 26, 2021 12:17pm",Comercio detallista,Ropa / Accesorios,01299,Bogotá D.C.,"mar. 26, 2021 12:42pm",6542000,1240000,30
1,008LJH6YHN,"feb. 10, 2021 3:31pm","feb. 10, 2021 3:15pm",Belleza y cuidado personal,Spa,00591,Bogotá D.C.,"feb. 10, 2021 3:34pm",4380000,1102000,10
2,00B3DGV62G,"mar. 3, 2021 3:26pm","mar. 3, 2021 3:09pm",Comercio detallista,Auto partes,02130,Bogotá D.C.,"mar. 3, 2021 3:30pm",11892500,1297000,65
3,00ILW1BB3Y,"mar. 24, 2021 12:45pm","mar. 24, 2021 12:33pm",Comercio detallista,Tienda naturista,00662,Bogotá D.C.,"mar. 24, 2021 12:51pm",372500,74500,8
4,01AK7UX4UU,"mar. 30, 2021 1:50pm","mar. 30, 2021 1:40pm",Comercio detallista,Tecnología,01784,Mosquera,"mar. 30, 2021 1:52pm",165500,0,4
...,...,...,...,...,...,...,...,...,...,...,...
3066,ZYU8UUM32R,"mar. 11, 2021 11:24am","mar. 11, 2021 11:14am",Comercio detallista,Tecnología,02829,Bogotá D.C.,"mar. 11, 2021 11:30am",26110920,4912000,18
3067,ZYV1ZCZ7IM,"mar. 25, 2021 2:46pm","mar. 25, 2021 2:36pm",Servicios,Tecnología y software,01348,Bogotá D.C.,"mar. 25, 2021 2:47pm",4400500,491000,11
3068,ZZJ3KWF9VC,"mar. 19, 2021 4:09pm","mar. 19, 2021 4:00pm",Alimentos y bebidas,Restaurante / Café,02938,Bogotá D.C.,"mar. 19, 2021 4:12pm",65100,20000,5
3069,ZZMG2NE0Q6,"mar. 12, 2021 11:05am","mar. 12, 2021 10:49am",Comercio detallista,Elementos de seguridad,01908,Bogotá D.C.,"mar. 12, 2021 11:06am",300000,149000,4


In [43]:
# Normalizar los datos
scaler = StandardScaler()
features = ["Monto ventas Acumulado", "Monto ventas ultimo mes", "cantidad transacciones"]
comercios_scaled = scaler.fit_transform(comercios[features]) 

In [44]:
comercios_scaled

array([[ 0.29439921,  0.38476588,  0.14920397],
       [ 0.00945575,  0.29360372, -0.29765129],
       [ 0.99957496,  0.42241981,  0.93120068],
       ...,
       [-0.55923181, -0.42116042, -0.40936511],
       [-0.52827288, -0.33594363, -0.43170787],
       [-0.39977155, -0.3111713 ,  0.59605923]])

In [45]:

# Aplicar K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
comercios["Cluster"] = kmeans.fit_predict(comercios_scaled)
    

In [46]:

# Crear la app Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard de Segmentación de Comercios"),
    dcc.Dropdown(
        id="cluster-selector",
        options=[{"label": f"Cluster {i}", "value": i} for i in range(4)],
        multi=True,
        value=[0, 1, 2, 3],
        placeholder="Seleccionar Clusters"
    ),
    dcc.Graph(id="scatter-plot")
])

@app.callback(
    Output("scatter-plot", "figure"),
    Input("cluster-selector", "value")
)
def update_graph(selected_clusters):
    filtered_df = comercios[comercios["Cluster"].isin(selected_clusters)]
    fig = px.scatter(filtered_df, x="Monto ventas Acumulado", y="Monto ventas ultimo mes",
                     color=filtered_df["Cluster"].astype(str), hover_data=["Id_comercio"])
    return fig
    

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, host="127.0.0.1", port=8050)